In [ ]:
from rl_helper.envs import create_vectorized_envs
from rl_helper.envs.pointmass.pointmass_env import PointMassParams
import torch.nn as nn
import torch
from torch.distributions import Normal

In [ ]:
envs = create_vectorized_envs("PointMass-v0", 128, params=PointMassParams(normalize_control=True))

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(0.1 * torch.randn(2, 2))
        # self.weight = nn.Parameter(torch.tensor([1.0, 0.0]).view(state_dim, 1))
        # self.weight = nn.Parameter(torch.zeros(state_dim, 1))
        self.logstd = nn.Parameter(torch.zeros(1, 2))

    def forward(self, state):
        print(state.shape)
        mean = state @ self.weight
        logstd = self.logstd.expand_as(mean)
        return Normal(mean, logstd.exp())

In [ ]:
def evaluate(num_eval_episodes, policy, envs):
    obs = envs.reset()
    dists_to_goal = []
    for eval_i in range(num_eval_episodes):
        action_distrib = policy(obs)
        use_action = action_distrib.mean
        obs, _, info, done = envs.step(use_action)
        for done_i in torch.nonzero(done):
            dists_to_goal.append(info['dist_to_goal'])
    return np.mean(dists_to_goal)

In [ ]:
policy = Policy()
evaluate(10, policy, envs)

In [ ]:
num_steps = 128
num_updates = 1000

for update_i in range(num_updates):
    for step_idx in range(cfg.num_steps):
        with torch.no_grad():
            act_data = policy.act(
                storage.get_obs(step_idx),
                storage.recurrent_hidden_states[step_idx],
                storage.masks[step_idx],
            )
        next_obs, reward, done, info = envs.step(act_data["action"])
        storage.insert(next_obs, reward, done, info, **act_data)
        logger.collect_env_step_info(info)

    updater.update(policy, storage, logger)
